In [1]:
import os
import numpy as np
from PIL import Image, ImageDraw
import torch
from torchvision import models, transforms
from sklearn.metrics.pairwise import cosine_similarity

# ==== CONFIG ====
LOGO_PATH = "upload/Bkav_logo.jpg"              # Đường dẫn logo mẫu
IMAGE_FOLDER = "images_to_check"              # Folder chứa ảnh cần kiểm tra
OUTPUT_FOLDER = "matched_output"              # Nơi lưu ảnh có vùng nghi là logo
WINDOW_SIZE = 128                              # Sliding window size (pixel)
STRIDE = 64                                    # Bước nhảy khi duyệt vùng
SIMILARITY_THRESHOLD = 0.75                    # Ngưỡng cosine similarity

# ==== TẠO THƯ MỤC KẾT QUẢ ====
os.makedirs(OUTPUT_FOLDER, exist_ok=True)

# ==== KHỞI TẠO MÔ HÌNH RESNET EMBEDDING ====
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
resnet = models.resnet50(pretrained=True).to(device)
embedding_model = torch.nn.Sequential(*list(resnet.children())[:-1])
embedding_model.eval()

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

def get_embedding(pil_image):
    img_tensor = transform(pil_image).unsqueeze(0).to(device)
    with torch.no_grad():
        features = embedding_model(img_tensor)
    return features.squeeze().cpu().numpy()

# ==== TÍNH EMBEDDING LOGO GỐC ====
print(f"🔍 Tính đặc trưng cho logo gốc: {LOGO_PATH}")
target_logo = Image.open(LOGO_PATH).convert("RGB")
target_embedding = get_embedding(target_logo)

# ==== DUYỆT QUA ẢNH TRONG THƯ MỤC ====
image_files = [f for f in os.listdir(IMAGE_FOLDER) if f.lower().endswith((".jpg", ".jpeg", ".png"))]
print(f"\n🔍 Đang kiểm tra {len(image_files)} ảnh trong thư mục '{IMAGE_FOLDER}'...\n")

for filename in image_files:
    path = os.path.join(IMAGE_FOLDER, filename)
    image = Image.open(path).convert("RGB")
    width, height = image.size
    draw = ImageDraw.Draw(image)
    match_found = False

    for top in range(0, height - WINDOW_SIZE + 1, STRIDE):
        for left in range(0, width - WINDOW_SIZE + 1, STRIDE):
            box = (left, top, left + WINDOW_SIZE, top + WINDOW_SIZE)
            region = image.crop(box)
            region_embedding = get_embedding(region)
            similarity = cosine_similarity(
                target_embedding.reshape(1, -1),
                region_embedding.reshape(1, -1)
            )[0][0]

            if similarity > SIMILARITY_THRESHOLD:
                draw.rectangle(box, outline="green", width=2)
                draw.text((left, top - 10), f"{similarity:.2f}", fill="green")
                match_found = True

    if match_found:
        save_path = os.path.join(OUTPUT_FOLDER, filename)
        image.save(save_path)
        print(f"✔ {filename}: phát hiện vùng nghi có logo")
    else:
        print(f"✘ {filename}: không phát hiện vùng nào")

print("\n✅ Hoàn tất. Kết quả lưu tại:", OUTPUT_FOLDER)

c:\Users\AnDVb\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\AnDVb\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


🔍 Tính đặc trưng cho logo gốc: upload/Bkav_logo.jpg

🔍 Đang kiểm tra 15 ảnh trong thư mục 'images_to_check'...

✔ test1.jpg: phát hiện vùng nghi có logo
✔ test10.jpg: phát hiện vùng nghi có logo
✘ test11.jpg: không phát hiện vùng nào
✘ test12.jpg: không phát hiện vùng nào
✔ test13.jpg: phát hiện vùng nghi có logo
✔ test14.jpg: phát hiện vùng nghi có logo
✘ test15.jpg: không phát hiện vùng nào
✔ test2.jpg: phát hiện vùng nghi có logo
✔ test3.jpg: phát hiện vùng nghi có logo
✔ test4.jpg: phát hiện vùng nghi có logo
✔ test5.jpg: phát hiện vùng nghi có logo
✔ test6.jpg: phát hiện vùng nghi có logo
✔ test7.jpg: phát hiện vùng nghi có logo
✔ test8.jpg: phát hiện vùng nghi có logo
✔ test9.jpg: phát hiện vùng nghi có logo

✅ Hoàn tất. Kết quả lưu tại: matched_output


In [3]:
from minio import Minio

client = Minio(
    endpoint="localhost:9000",
    access_key="minioadmin",
    secret_key="minioadmin",
    secure=False
)

bucket_name = "logo-matching"
prefix = "images_to_check/"

print("📂 Danh sách ảnh trong MinIO:")
objects = client.list_objects(bucket_name, prefix=prefix, recursive=True)
for obj in objects:
    print("🖼️", obj.object_name)


📂 Danh sách ảnh trong MinIO:
🖼️ images_to_check/test1.jpg
🖼️ images_to_check/test10.jpg
🖼️ images_to_check/test11.jpg
🖼️ images_to_check/test12.jpg
🖼️ images_to_check/test13.jpg
🖼️ images_to_check/test14.jpg
🖼️ images_to_check/test15.jpg
🖼️ images_to_check/test2.jpg
🖼️ images_to_check/test3.jpg
🖼️ images_to_check/test4.jpg
🖼️ images_to_check/test5.jpg
🖼️ images_to_check/test6.jpg
🖼️ images_to_check/test7.jpg
🖼️ images_to_check/test8.jpg
🖼️ images_to_check/test9.jpg
